pretrained model : resnet18

loss_fn : CrossEntropyLoss

optimizer : adam

epoch: 5

batch-size : 32

data augmentation : transforms.RandomHorizontalFlip(),

fc layer : relu

lr : 0.0001

Normalized : (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)

 + alpha(더 시도해볼 것들)

seresnet50

epoch 증가 (거의 더 좋을 듯)

Normalized mean, std값 조정 -> 완료


In [31]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler

from torchvision import transforms, utils
from torchvision.transforms import Resize, ToTensor, Normalize

import time

In [32]:
train_path = 'input/data/train'
train_image_dir_path = os.path.join(train_path, 'images')

이하 CUDA out of memory. Tried to allocate 1.50 GiB (GPU 0; 31.75 GiB total capacity; 5.97 GiB already allocated; 1.23 GiB free; 6.98 GiB reserved in total by PyTorch)

떴을 때

In [33]:
# import gc
# gc.collect()
# torch.cuda.empty_cache()

Dataset 생성

모든 train data의 path를 가져와 라벨링 진행

In [34]:
def search(dirname, result): # 하위 목록의 모든 파일을 찾는 함수
    try:
        filenames = os.listdir(dirname)
        for filename in filenames:
            if filename[0] == '.': # .으로 시작하는 애들 거름
                continue
            full_filename = os.path.join(dirname, filename)
            if os.path.isdir(full_filename):
                search(full_filename, result)
            else:
                ext = os.path.splitext(full_filename)[-1] # 확장자 체크
                if ext:
                    result.append(full_filename)
        
    except PermissionError:
        print('Permission Error')
        pass

In [35]:
all_path = list()
search(train_image_dir_path, all_path)

train의 데이터 디렉토리는 2700개로, 각각의 이미지 파일(incorrect, mask1~5, normal)을 곱한 갯수가 나옵니다.

In [36]:
len(all_path) # 2700 * 7

18900

In [37]:
all_path[:10]

['input/data/train/images/001752_male_Asian_53/mask5.jpg',
 'input/data/train/images/001752_male_Asian_53/mask4.jpg',
 'input/data/train/images/001752_male_Asian_53/mask2.jpg',
 'input/data/train/images/001752_male_Asian_53/mask3.jpg',
 'input/data/train/images/001752_male_Asian_53/normal.jpg',
 'input/data/train/images/001752_male_Asian_53/mask1.jpg',
 'input/data/train/images/001752_male_Asian_53/incorrect_mask.jpg',
 'input/data/train/images/005127_female_Asian_51/mask5.jpg',
 'input/data/train/images/005127_female_Asian_51/mask4.jpg',
 'input/data/train/images/005127_female_Asian_51/mask2.jpg']

파일의 확장자는 jpg, png, jpeg로 3종류가 있습니다.

In [38]:
exts = list()
for word in all_path:
    ext = os.path.splitext(word)[-1]
    if ext not in exts:
        exts.append(ext)
print(exts) # jpg, png, jpeg

['.jpg', '.png', '.jpeg']


In [39]:
all_path = sorted(all_path)

라벨링을 하는 함수입니다. 조건에 따라 label에 숫자를 더해주는 식으로 만들었습니다.

In [40]:
def labeling(name):
    label = 0
    info, mask_type = name.split('/')[-2:]
    info = info.split('_')
    gender, age = info[1], int(info[3])
    
    # 마스크 구별
    if 'incorrect' in mask_type:
        label += 6
    elif 'normal' in mask_type:
        label += 12
    
    # gender 구별
    if gender == 'female':
        label += 3
    
    # 나이 구별
    if 30 <= age and age < 60:
        label += 1
    elif age >= 60:
        label += 2
    
    return label

path, label을 컬럼으로 갖는 dataframe을 생성해줍니다.

In [41]:
train_path_label = pd.DataFrame(all_path, columns = ['path'])

train_path_label['label'] = train_path_label['path'].map(lambda x: labeling(x))
train_path_label

,path,label
0,input/data/train/images/000001_female_Asian_45...,10
1,input/data/train/images/000001_female_Asian_45...,4
2,input/data/train/images/000001_female_Asian_45...,4
3,input/data/train/images/000001_female_Asian_45...,4
4,input/data/train/images/000001_female_Asian_45...,4
...,...,...
18895,input/data/train/images/006959_male_Asian_19/m...,0
18896,input/data/train/images/006959_male_Asian_19/m...,0
18897,input/data/train/images/006959_male_Asian_19/m...,0
18898,input/data/train/images/006959_male_Asian_19/m...,0


In [42]:
# train_path_label.to_csv('./train_path_label.csv', index=False, encoding='utf-8')
# train_path_label = pd.read_csv('./train_path_label.csv', encoding='utf-8')

dataset을 상속받아 만든 CustomDataset입니다. transform은 size를 [512, 384]로 변형하고, Tensor로 만들고, 정규화를 해주었습니다.

In [43]:
class CustomDataset(Dataset):
    def __init__(self, img_paths_label, transform):
        self.X = img_paths_label['path']
        self.y = img_paths_label['label']
        self.transform = transform
        
    def __getitem__(self, index):
        image = Image.open(self.X.iloc[index])
        label = self.y.iloc[index]
        
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label)
    
    def __len__(self):
        return len(self.X)

In [44]:
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    transforms.RandomHorizontalFlip(p=0.5),
    ToTensor(),
#     Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

/opt/conda/lib/python3.8/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


train, valid를 나누는 부분입니다.

label의 비율을 유지하면서 나눴습니다.

+ 기존 방법 대신StratifiedKFold 사용을 시도했습니다.

In [45]:
# 원본
# from sklearn.model_selection import train_test_split
# train, valid = train_test_split(train_path_label, test_size=0.2,
#                                shuffle=True, stratify=train_path_label['label'],
#                                random_state=34)

DataLoader
- index를 사용한 Dataloader 정의
- getDataloader 함수 설명
    1. Pytorch Dataset, train 인덱스, valid 인덱스, batch size를 전달받아 Train, Valid DataLoader 객체를 반환합니다.
    2. torch.utils.data.Subset 객체는 데이터셋과 해당 데이터셋의 인덱스를 전달받아 Subset 객체를 생성합니다. 생성한 Subset 객체를 사용해 DataLoader 객체를 반환합니다.

In [46]:
def getDataloader(dataset, train_idx, valid_idx, batch_size, num_workers):
    # 인자로 전달받은 dataset에서 train_idx에 해당하는 Subset 추출
    train_set = torch.utils.data.Subset(dataset,
                                        indices=train_idx)
    # 인자로 전달받은 dataset에서 valid_idx에 해당하는 Subset 추출
    val_set   = torch.utils.data.Subset(dataset,
                                        indices=valid_idx)
    
    # 추출된 Train Subset으로 DataLoader 생성
    train_loader = torch.utils.data.DataLoader(
        train_set,
        batch_size=batch_size,
        num_workers=num_workers,
        drop_last=True,
        shuffle=True
    )
    # 추출된 Valid Subset으로 DataLoader 생성
    val_loader = torch.utils.data.DataLoader(
        val_set,
        batch_size=batch_size,
        num_workers=num_workers,
        drop_last=True,
        shuffle=False
    )
    
    # 생성한 DataLoader 반환
    return train_loader, val_loader

dataloader를 정의했습니다. batchsize는 64로 했고 shuffle을 했습니다.

In [47]:
BATCH_SIZE = 32

In [48]:
dataset = CustomDataset(train_path_label, transform)

custom_dataloader = DataLoader(dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=True
                             )

모델

모델은 pretrain된 resnet18을 가져왔습니다. 이 모델의 마지막 fc층만 저희의 과제인 18개의 class로 변경해줍니다.

In [49]:
# !pip install --upgrade torch torchvision

In [50]:
my_model = torchvision.models.efficientnet_b7(pretrained=True)

In [51]:
# output 18개로
IN_FEATURES = my_model.classifier[1].in_features

OUTPUT_CLASS_NUM = 18
my_model.classifier[1] = torch.nn.Linear(in_features = IN_FEATURES, out_features = OUTPUT_CLASS_NUM, bias=True)  

In [52]:
import math

OUTPUT_CLASS_NUM = 18
my_model.fc = torch.nn.Linear(in_features=512, out_features=OUTPUT_CLASS_NUM, bias=True) # output 18개로

# xavier uniform
torch.nn.init.xavier_uniform_(my_model.fc.weight)
stdv = 1. / math.sqrt(my_model.fc.weight.size(1))
my_model.fc.bias.data.uniform_(-stdv, stdv)

my_model.fc.weight.shape[0]

18

In [53]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [54]:
# !pip install timm

In [55]:
# # my_model = torchvision.models.resnet18(pretrained=True)

# import timm
# my_model = timm.create_model('resnet50', pretrained=True)

In [56]:
# import math

# OUTPUT_CLASS_NUM = 18
# print(my_model.fc)
# num_ftrs = my_model.fc.in_features
# print(num_ftrs)
# # my_model.fc = torch.nn.Linear(in_features=num_ftrs, out_features=OUTPUT_CLASS_NUM, bias=True) # output 18개로

# my_model.fc = nn.Sequential(
#                    nn.Linear(num_ftrs, 512),
#                    nn.ReLU(inplace=True),
#                    nn.Dropout(0.2),
#                    nn.Linear(512, 512),
#                    nn.ReLU(inplace=True),
#                    nn.Dropout(0.2),
#                    nn.Linear(512, OUTPUT_CLASS_NUM)
#                 ).to(device)

# # Linear, ReLU, Dropout 넣기

# # num_ftrs = my_model.fc.in_features
# # my_model.classifier = nn.Sequential(
# #     nn.Linear(512 * 7 * 7, 4096),
# #     nn.ReLU(True),
# #     nn.Dropout(p=0.2),
# #     nn.Linear(4096, 4096),
# #     nn.ReLU(True),
# #     nn.Dropout(p=0.2),
# #     nn.Linear(4096, OUTPUT_CLASS_NUM),
# # )



# # xavier uniform
# for i in range(0, len(my_model.fc), 3):
#     torch.nn.init.xavier_uniform_(my_model.fc[i].weight)
#     stdv = 1. / math.sqrt(my_model.fc[i].weight.size(1))
#     my_model.fc[i].bias.data.uniform_(-stdv, stdv)

# my_model.fc[-1].weight.shape[0]

다음 코드를 참고하세요.
- pre-trained model에 원하는 함수 추가
- 입력값 모를 때 자동으로 계산해서 넣는 법

In [57]:
# pre-trained model에 원하는 함수 추가

# num_classes = 18
# model = vgg19_bn(pretrained=True)
# model.classifier = nn.Sequential(
#     nn.Linear(512 * 7 * 7, 4096),
#     nn.ReLU(True),
#     nn.Dropout(),
#     nn.Linear(4096, 4096),
#     nn.ReLU(True),
#     nn.Dropout(),
#     nn.Linear(4096, num_classes),
# )

# model

In [58]:
# 입력값 모를 때 넣는 법

# model_conv = torchvision.models.resnet18(pretrained=True)
# num_ftrs = model_conv.fc.in_features
# model_conv.fc = nn.Linear(num_ftrs, 2)

In [59]:
# 임의로 둘을 합쳐봄

# from torchvision.models import vgg19_bn

# OUTPUT_CLASS_NUM = 18
# model = vgg19_bn(pretrained=True)
# num_ftrs = model.fc.in_features
# model.classifier = nn.Sequential(
#     nn.Linear(num_ftrs, 512),
#     nn.ReLU(True),
#     nn.Dropout(p=0.2),
#     nn.Linear(512, 512),
#     nn.ReLU(True),
#     nn.Dropout(p=0.2),
#     nn.Linear(512, OUTPUT_CLASS_NUM),
# )

# model

아래 대부분의 코드가 부스트캠프에서 학습 자료나 과제로 제공받았던 코드를 거의 그대로 사용했습니다.

설명도 주석도 잘 달려 있어서 그대로 가져왔습니다.

epoch는 5, lr은 0.0001로 주었습니다.

추후에 lr scheduler로 lr을 변경해보는 방법도 좋을 것 같습니다.

In [60]:
my_model.to(device)

LEARNING_RATE = 0.0001 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 5 # 학습 때 mnist train data set을 얼마나 많이 학습할 지 결정하는 옵션

loss_fn = torch.nn.CrossEntropyLoss() # 분류 학습 때 많이 사용되는 Cross Entropy Loss를 objective function으로 사용
# loss_fn = torch.nn.NLLLoss() # VGG
optimizer = torch.optim.Adam(my_model.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함


In [61]:
# 5-fold Stratified KFold 5개의 fold를 형성하고 5번 Cross Validation을 진행합니다.
from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import KFold

best_test_accuracy = 0.
best_test_loss = 9999.
start = time.time()  # 시작 시간 저장

# skf 설정
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits)
# kfold = KFold(n_splits=4, shuffle=False)

# skf에서 사용할 labels 설정
labels = [i for i in train_path_label['label']]

BATCH_SIZE = 32
num_workers = 4

print(train_path_label)
temp_idx = 0
for train_index, validate_index in skf.split(train_path_label, labels):
#     print(train_index, validate_index)
    temp_idx += 1
    train = train_path_label.iloc[train_index]
    valid = train_path_label.iloc[validate_index]

    train_dataset = CustomDataset(train, transform)

    train_dataloader = DataLoader(train_dataset,
                                 batch_size=BATCH_SIZE,
                                 shuffle=False
                                 )

    valid_dataset = CustomDataset(valid, transform)

    valid_dataloader = DataLoader(valid_dataset,
                                  batch_size=BATCH_SIZE,
                                  shuffle=False)

    dataloaders = {
        "train": train_dataloader,
        "test": valid_dataloader,
    }
    for epoch in range(NUM_EPOCH):

        for phase in ["train", "test"]:
            running_loss = 0.
            running_acc = 0.
#             # 네트워크 모델을 train 모드로 두어 gradient를 계산하고, 
#             # 여러 sub module (배치 정규화, 드롭아웃 등)이 train_mode로 작동할 수 있게 함.
            if phase == "train":
                my_model.train()
            # 네트워크 모델을 eval 모드로 두어 여러 sub module들이 eval mode로 작동할 수 있게 함.
            elif phase == "test":
                my_model.eval()
            
            for ind, (images, labels) in enumerate(dataloaders[phase]):
                images = images.to(device)
                labels = labels.to(device)

                optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함.

                # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
                with torch.set_grad_enabled(phase == "train"):
                    logits = my_model(images)
                    # 모델에서 linear 값으로 나오는 예측 값([0.9, 1.2, 3.2, 0.1, -0.1, ...])에서 최대 output index를 찾아 예측 레이블([2])로 변경함
                    _, preds = torch.max(logits, 1)
                    loss = loss_fn(logits, labels)

                    if phase == "train":
                        loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient를 계산
                        optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

                running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
                running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장

            # 한 epoch이 모두 종료되었을 때,
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_acc / len(dataloaders[phase].dataset)

            seconds = int(time.time() - start)
            print(f"현재 epoch-{temp_idx}-{epoch}의 {phase}-데이터 셋에서 평균 Loss: {epoch_loss:.3f}, 평균 Accuracy: {epoch_acc:.3f}")
            print(f"소요 시간: {seconds // 60}분 {seconds % 60}초")  # 현재시각 - 시작시간 = 실행 시간
#             print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss: {epoch_loss:.3f}, 평균 Accuracy: {epoch_acc:.3f}")
#             print(f"소요 시간: {seconds // 60}분 {seconds % 60}초")  # 현재시각 - 시작시간 = 실행 시간

            # phase가 test일 때
            if phase == "test":
                # best accuracy 계산
                if best_test_accuracy < epoch_acc:
                    best_test_accuracy = epoch_acc
                # best loss 계산
                if best_test_loss > epoch_loss:
                    best_test_loss = epoch_loss

                
seconds = int(time.time() - start)
print("학습 종료!")
print(f"최고 accuracy: {best_test_accuracy}, 최고 낮은 loss: {best_test_loss}")
print(f"소요 시간: {seconds // 60}분 {seconds % 60}초")  # 현재시각 - 시작시간 = 실행 시간

                                                    path  label
0      input/data/train/images/000001_female_Asian_45...     10
1      input/data/train/images/000001_female_Asian_45...      4
2      input/data/train/images/000001_female_Asian_45...      4
3      input/data/train/images/000001_female_Asian_45...      4
4      input/data/train/images/000001_female_Asian_45...      4
...                                                  ...    ...
18895  input/data/train/images/006959_male_Asian_19/m...      0
18896  input/data/train/images/006959_male_Asian_19/m...      0
18897  input/data/train/images/006959_male_Asian_19/m...      0
18898  input/data/train/images/006959_male_Asian_19/m...      0
18899  input/data/train/images/006959_male_Asian_19/n...     12

[18900 rows x 2 columns]


RuntimeError: CUDA out of memory. Tried to allocate 384.00 MiB (GPU 0; 31.75 GiB total capacity; 1.06 GiB already allocated; 32.50 MiB free; 1.09 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
print(train)

In [ ]:
# train[0].shape()

dataloader를 정의했습니다. batchsize는 64로 했고 shuffle을 했습니다.

In [ ]:
PATH = '/input'

torch.save(my_model, PATH)

In [ ]:
print(my_model)

검증

In [ ]:
def func_eval(model, data_iter, device):
    with torch.no_grad():
        n_total, n_correct = 0, 0
        model.eval()
        for batch_in, batch_out in data_iter:
            y_trgt = batch_out.to(device)
            model_pred = model.forward(batch_in.to(device))
            _, y_pred = torch.max(model_pred, 1)  # 행으로 비교
            n_correct += (y_pred == y_trgt).sum().item()
            n_total += batch_in.size(0)
        val_acc = (n_correct/n_total)
        #model_train()
    return val_acc

In [ ]:
func_eval(my_model, valid_dataloader, device)

In [ ]:
def check_eval(raw_data, dataloader, model, device):
    result = []
    with torch.no_grad():
        model.eval()
        for i, (X,y) in enumerate(dataloader):
            model_pred = model.forward(X.to(device))
            _, y_pred = torch.max(model_pred, 1)
            
            result.append([valid.iloc[i]['path'], y_pred.cpu().numpy()[0], y.cpu().numpy()[0]])
    result = pd.DataFrame(result, columns=['path', 'pred', 'target'])
    return result

In [ ]:
valid_testing_dataloader = DataLoader(valid_dataset, shuffle=False)

check_eval_df = check_eval(valid, valid_testing_dataloader, my_model, device)
check_eval_df

잘못 예측한 데이터 리스팅

In [ ]:
wrong_df = check_eval_df[check_eval_df['pred'] != check_eval_df['target']]
wrong_df = wrong_df.reset_index(drop=True)
wrong_df

In [ ]:
class_list = [[['wear'], ['male'], ['un30']], [['wear'], ['male'], ['3060']], [['wear'], ['male'], ['ov60']],
              [['wear'], ['fema'], ['un30']], [['wear'], ['fema'], ['3060']], [['wear'], ['fema'], ['ov60']],
              [['Inco'], ['male'], ['un30']], [['Inco'], ['male'], ['3060']], [['Inco'], ['male'], ['ov60']],
              [['Inco'], ['fema'], ['un30']], [['Inco'], ['fema'], ['3060']], [['Inco'], ['fema'], ['ov60']],
              [['NoWe'], ['male'], ['un30']], [['NoWe'], ['male'], ['3060']], [['NoWe'], ['male'], ['ov60']],
              [['NoWe'], ['fema'], ['un30']], [['NoWe'], ['fema'], ['3060']], [['NoWe'], ['fema'], ['ov60']]
             ]

def draw_(df):
    plt.figure(figsize = (15, 30))
    row = len(wrong_df) // 3
    # 틀린 번호 찾기
    wrong_number = list()
    for df_path in list(df['path']):
        wrong_number.append(df_path.split('/')[4].split('_')[0])
        
    for i in range(df.shape[0]):
        plt.subplot(row + 1, df.shape[0] // row, i + 1)
        plt.imshow(Image.open(df['path'][i]))
        plt.title(f"target:{class_list[df['target'][i]]}, \n pred: {class_list[df['pred'][i]]} \n id: {wrong_number[i]}", color='r', size=20)
        plt.axis('off')
    print(df)
    

    plt.tight_layout()
    plt.show()

In [ ]:
print(wrong_df)

In [ ]:
# draw_(wrong_df)

f1 score 측정

In [ ]:
# !pip install seaborn

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, classification_report

함수 정의
- precision, recall, f1-score, confusion matrix 출력

In [ ]:
def report():
    y_test, y_pred = check_eval_df['target'], check_eval_df['pred']
    print(classification_report(y_test, y_pred))
    print('my_model Accuracy:', np.mean(y_pred == y_test))
    ax = plt.figure(figsize=(10, 5))
    sns.heatmap(confusion_matrix(y_test, y_pred), linewidths=0.5, fmt='.1f', annot=True)

In [ ]:
report()

데이터 저장

In [ ]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [ ]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = 'input/data/eval'

In [ ]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    transforms.RandomHorizontalFlip(p=0.5),
    ToTensor(),
#     Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),

])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
# model = MyModel(num_classes=18).to(device)
PATH = '/input'
model = torch.load(PATH)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

In [ ]:
test_data = pd.read_csv('input/data/eval/submission.csv', encoding='utf-8')

In [ ]:
classes = dict()

for cla in test_data['ans']:
    if cla in classes:
        classes[cla] += 1
    else:
        classes[cla] = 1
        
classes = sorted(classes.items())
print(classes)